In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

plt.rcParams['figure.figsize'] = (7,7) # Make the figures a bit bigger

from keras.datasets import mnist
from keras.models import Sequential,load_model

from keras.layers.core import Dense, Dropout, Activation,Lambda,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

from keras.utils import np_utils


C:\MasterSoftware\anaconda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
nb_classes = 10
# the data, shuffled and split between tran and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

np.save('C:/Master/Thesis/code/testdata',X_test,y_test)

X_train = X_train.reshape(X_train.shape[0], 28, 28,1)  
X_test = X_test.reshape(X_test.shape[0],28, 28,1)  

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


X_train /= 255
X_test /= 255    # why do I need this two line, when i delete it, accuracy becomes 0.29

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)
print(type(X_test))


Y_train = np_utils.to_categorical(y_train, nb_classes) # convert a class vector y_train to a binary matrix with size nb_class
Y_test = np_utils.to_categorical(y_test, nb_classes) #e.g. convert y=3 to [0,0,1,0,0,0...]


X_train original shape (60000, 28, 28)
y_train original shape (60000,)
X_test original shape (10000, 28, 28)
y_test original shape (10000,)
Training matrix shape (60000, 28, 28, 1)
Testing matrix shape (10000, 28, 28, 1)
<class 'numpy.ndarray'>


In [2]:
sess = tf.InteractiveSession()


In [13]:

#sess = tf.InteractiveSession()
#pdb.set_trace()
x=tf.placeholder(tf.float32, (1,1,1,32))
x_2dim =K.reshape(x,[-1,K.shape(x)[3]])
k =3
values, indices = tf.nn.top_k(x_2dim, k, sorted=False)
my_range = K.expand_dims(K.arange(0, K.shape(x_2dim)[0]), 1)  # will be [[0], [1]],,indices.get_shape()[0]
my_range_repeated = K.tile(my_range, [1, k])  # will be [[0, 0], [1, 1]]
a1 = K.expand_dims(my_range_repeated, 2)
a2 = K.expand_dims(indices, 2)
full_indices = tf.concat(axis=2, values=[a1, a2])
full_indices2 = K.reshape(full_indices, [-1, 2])#-1 means I don't know the value here, the program calculate it
c = K.cast(K.shape(x_2dim),dtype = 'int32')
z = K.reshape(c,[-1])
pp= tf.reshape(values, [-1])

output = tf.scatter_nd(full_indices2, pp, z)
#output = tf.sparse_to_dense(full_indices2, z, pp , default_value=0, validate_indices = False)
# sparse_tensor = tf.SparseTensor(full_indices2, pp, z)
# output = tf.sparse_add(tf.zeros(z), sparse_tensor)
final_0 = K.expand_dims(output,1)
final_1 = K.expand_dims(final_0,1)
final = K.cast(final_1,dtype = 'float32')    


In [15]:
i=np.random.randn(1,1,1,32)
#i=np.zeros((1,1,1,32))
i[0,0,0,10]=10
print(i)
print(output.eval({x:i}))
if tf.gradients(output,x)[0] is None:
    print("is None")

print(tf.gradients(output,x)[0].eval({x:i}))

# print(sess.eval([x_2dim],feed_dict={x:i}))

[[[[ 0.19524364  1.06517884 -0.29040163  0.66727467  0.3545482
     0.56784926  2.32782646 -0.27062221  1.09262009  0.63597782
    10.          0.72470761  0.37102081  1.10599569  0.45035431
    -0.85874841  0.36217118 -0.90888626 -0.96927774  0.92603825
     0.20556381  0.67898866  1.77274911  0.18761335 -0.44831088
     0.78853091  0.58251262 -0.30420721  0.40493667  0.66810422
     0.38778011 -1.08062276]]]]
[[ 0.         0.         0.         0.         0.         0.
   2.3278265  0.         0.         0.        10.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         1.7727491  0.
   0.         0.         0.         0.         0.         0.
   0.         0.       ]]
[[[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
    1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]]


In [13]:
print('after fit')
model.save('C:/Master/Thesis/code/my_model.h5')
output = model.predict(X_test[0].reshape(1,28,28,1), batch_size=1, verbose=0)

after fit
